In [395]:
'''
t = '2021/12/10'
calibrated on options of S&P500 index

'''

"\nt = '2021/12/10'\ncalibrated on options of S&P500 index\n\n"

In [394]:
import yfinance as yf
import pandas as pd
from scipy.interpolate import CubicSpline
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [362]:
t = datetime.strptime('2021-12-10', '%Y-%m-%d').date()
spx = yf.Ticker('^SPX')

In [255]:
opt = spx.option_chain(date='2021-12-15')

In [258]:
spot = 4712.02

In [320]:
def otmCalls_function(calls):
    otmCalls = calls[~calls.inTheMoney]
    otmCalls = otmCalls[['strike', 'impliedVolatility']]
    otmCalls.reset_index(drop=True, inplace=True)
    min = 0.0
    for i in otmCalls.index:
        [strike, impliedVolatility] = otmCalls.loc[i]
        if impliedVolatility > min:
            min = impliedVolatility
        else:
            otmCalls = otmCalls.drop([i])
    return otmCalls

In [321]:
def otmPuts_function(puts):
    otmPuts = puts[~puts.inTheMoney]
#df[(df.line_race != 0) & (df.line_race != 10)]
#df.drop(df.index[df['myvar'] == 'specific_name'], inplace = True)
    otmPuts = otmPuts[['strike', 'impliedVolatility']]
#otmPuts_new = otmPuts[(otmPuts.impliedVolatility != 0.2500075) & (otmPuts.impliedVolatility != 0.500005)]
#otmPuts_new.shape
    otmPuts.reset_index(drop=True, inplace=True)
    max = otmPuts.iloc[0,1]
    for i in otmPuts.index:
        [strike, impliedVolatility] = otmPuts.loc[i]
        if impliedVolatility < max:
            max = impliedVolatility
        else:
            otmPuts = otmPuts.drop([i])
    return otmPuts

In [292]:
#remove outliers
'''import scipy.stats as stats
z = np.abs(stats.zscore(data))
data_clean = data[(z<2).all(axis=1)]
data_clean.shape'''

'import scipy.stats as stats\nz = np.abs(stats.zscore(data))\ndata_clean = data[(z<2).all(axis=1)]\ndata_clean.shape'

In [318]:
dates = spx.options
dates_list = list(dates)
dates_list.pop(0)
dates_list
#base = datetime.datetime.today()
#date_list = [base - datetime.timedelta(days=x) for x in range(numdays)]
dates_test = ['2021-12-31', '2022-01-31', '2022-02-28', '2022-03-31', '2022-04-29', '2022-05-31', '2022-06-30']
dates_test

['2021-12-31',
 '2022-01-31',
 '2022-02-28',
 '2022-03-31',
 '2022-04-29',
 '2022-05-31',
 '2022-06-30']

In [304]:
def makeSmile(dates_list):
    volSurface = {}
    for i in range(10):
        opt = spx.option_chain(date=date)
        calls = opt.calls
        puts = opt.puts
        otmCalls = otmCalls_function(calls)
        otmPuts = otmPuts_function(puts)
        data = pd.concat([otmPuts, otmCalls], ignore_index=True)
        volSurface[date] = CubicSpline(data.strike, data.impliedVolatility, bc_type = 'natural')
    return volSurface

In [326]:
volSurface = makeSmile(dates_test)

<function dict.keys>

In [391]:
from datetime import datetime
def makeSurface(volSurface, T, K):
    dates = list(volSurface.keys())
    if T not in dates:
        vol = []
        dates.append(T)
        dates.sort()
        index = dates.index(T)
        T1_index, T2_index = dates[index-1], dates[index+1]
        T1 = datetime.strptime(T1_index, '%Y-%m-%d').date()
        T2 = datetime.strptime(T2_index, '%Y-%m-%d').date()
        T_date = datetime.strptime(T, '%Y-%m-%d').date()
        vol1 = volSurface[T1_index]
        vol2 = volSurface[T2_index]
        for strike in K:
            factor_1 = (T2-T_date)/(T2-T1)*vol1(strike)**2*(T1-t).days
            factor_2 = (T_date-T1)/(T2-T1)*vol2(strike)**2*(T2-t).days
            vol.append(np.sqrt((factor_1 + factor_2)/(T_date-t).days))
        volSurface[T] = CubicSpline(K, vol, bc_type = 'natural')
    return volSurface

In [393]:
K = list(range(3500, 5500, 100))
T = '2022-01-15'
makeSurface(volSurface, T, K)

{'2021-12-31': <scipy.interpolate._cubic.CubicSpline at 0x22df731e5e0>,
 '2022-01-31': <scipy.interpolate._cubic.CubicSpline at 0x22df72b3360>,
 '2022-02-28': <scipy.interpolate._cubic.CubicSpline at 0x22df72bd1d0>,
 '2022-03-31': <scipy.interpolate._cubic.CubicSpline at 0x22df72e0680>,
 '2022-04-29': <scipy.interpolate._cubic.CubicSpline at 0x22df7324950>,
 '2022-05-31': <scipy.interpolate._cubic.CubicSpline at 0x22df72cb630>,
 '2022-06-30': <scipy.interpolate._cubic.CubicSpline at 0x22df72c2590>,
 '2022-01-15': <scipy.interpolate._cubic.CubicSpline at 0x22df72fd950>}